In [ ]:
import matplotlib.pyplot as plt
%matplotlib notebook
import matplotlib as mpl
import numpy as np
import seaborn as sns

In [ ]:
baseFolder = "/simdata/on74yces/merge_E1_flat_new"

In [ ]:
def readSliceData(fileName):
    with open(fileName) as f:
        line = f.readline()[2:].split(" ")
        nx = int(line[0])
        nz = int(line[1])
    svf = np.loadtxt(fileName)
    return svf.reshape((nz,nx)).transpose() # x: dim0, z: dim1

def extractMaxAvailableTimeStep():
    import glob

    baseFileName = "/svfSlice_"
    fileEnding = ".txt"
    allFiles = glob.glob(baseFolder + baseFileName+"*"+fileEnding)
    maxTimeStep = np.max([int(f[(f.find(baseFileName)+len(baseFileName)):f.find(fileEnding)]) for f in allFiles])
    return maxTimeStep

def getCurrentForcing(t):
    fileName = baseFolder+"/fluidInfo.txt"
    # columns: t, fx, uMean, ...
    content = np.loadtxt(fileName).transpose()
    idx = np.argwhere(content[0] == t)
    return content[1][idx]
        

In [ ]:
# source: 
# https://stackoverflow.com/questions/26563858/matplotlib-imshow-fixed-aspect-and-vertical-colorbar-matching-master-axis-height
def match_colorbar(cb, ax=None):
    """
    Match the size of the colorbar with the size of the axes.
    
    Args:
        ax: Axes from which the colorbar "stole" space.
        cb: Colorbar to match to `ax`.
    """
    ax = ax or plt.gca()
    bbox = ax.get_position()
    cb_bbox = cb.ax.get_position()
    if cb.orientation == "vertical":
        # Update bottom and height.
        left = cb_bbox.xmin
        width = cb_bbox.width
        bottom = bbox.ymin
        height = bbox.height
    else:
        # Update left and width.
        left = bbox.xmin
        width = bbox.width
        bottom = cb_bbox.ymin
        height = cb_bbox.height
    pos = [left, bottom, width, height]
    cb.ax.set_position(pos)

In [ ]:
def evalTimeStep(t):
    
    bedHeightThreshold = 0.3
    
    timestep = str(t)
    fileName = baseFolder+"/svfSlice_"+timestep+".txt"
    svf = readSliceData(fileName)
    fileName = baseFolder+"/fillSlice_"+timestep+".txt"
    fill = readSliceData(fileName)
    fileName = baseFolder+"/velXSlice_"+timestep+".txt"
    vel = readSliceData(fileName)
    
    fx = getCurrentForcing(t)
    
    #print("Total water volume: " , np.sum(fill))
    
    #plt.figure()
    #plt.imshow(svf<bedHeightThreshold, interpolation='none')
    #plt.imshow(vel, interpolation='none')
    #plt.colorbar()
    #plt.show()
    
    xlen = svf.shape[0]
    zlen = svf.shape[1]


    xPos = np.linspace(0.5, xlen - 0.5, num = xlen)
    zPos = np.linspace(0.5, zlen - 0.5, num = zlen)

    waterHeightOverX = np.zeros(xlen)
    bedHeightOverX = np.zeros(xlen)
    froudeOverX = np.zeros(xlen)
    shieldsOverX = np.zeros(xlen)
    ReFricOverX = np.zeros(xlen)
    
    #plt.figure()
    
    avgUMean = 0
    avgWaterHeight = 0

    for x in range(xlen):
        svfX = svf[x,:]
        fillX = fill[x,:]
        velX = vel[x,:]

        waterElevation = np.sum(fillX)
        idxWaterElevation = int(waterElevation)
        
        idxBedHeight = zlen - np.argmax(svfX[::-1]>bedHeightThreshold) - 1
        bedHeight = zPos[idxBedHeight] + (bedHeightThreshold - svfX[idxBedHeight]) / (svfX[idxBedHeight+1] - svfX[idxBedHeight]) * 1

        waterHeight = waterElevation - bedHeight
        
        #print(idxWaterElevation, idxBedHeight, svfX[idxBedHeight+1], svfX[idxBedHeight], svfX[idxBedHeight-1])

        uMean = np.average(velX[idxBedHeight:idxWaterElevation+1])
        
        #velDerivative = np.gradient(velX)
        # velocity derivative is very sensitive to the exact location -> where to take it?
        # here: take average over some cells right above the bed
        #velDerivativeAtBed = np.average(velDerivative[idxBedHeight+1:idxBedHeight+5]) # TODO check this!!!!
        #wallShearStress = dynViscosity * velDerivativeAtBed 
        # note: this does only get a small part of the total stress on the particle -> different approach necessary
        
        # integrate current forcing over water height to the position of the bed (see discussion with Bernhard)
        wallShearStress = waterHeight * fx
        
        frictionVelocity = np.sqrt(wallShearStress / fluidDensity)

        waterHeightOverX[x] = waterHeight
        bedHeightOverX[x] = bedHeight
        froudeOverX[x] = uMean / np.sqrt(waterHeight * gravAcceleration)
        shieldsOverX[x] = wallShearStress / ((densityRatio - 1) * gravAcceleration * avgDiameter)
        ReFricOverX[x] = frictionVelocity * waterHeight / kinematicViscosity
        
        
    
        #plt.plot(velDerivative[idxBedHeight:idxWaterElevation-1])
        #plt.plot(velX[idxBedHeight:idxWaterElevation+1])
        
        avgWaterHeight += waterHeight
        avgUMean += uMean
        
    #plt.xlabel("height")
    #plt.ylabel("velX")
    #plt.show()
    
    avgWaterHeight /= float(xlen)
    avgUMean /= float(xlen)
    
    ReBulk = avgUMean * avgWaterHeight / kinematicViscosity
    
    print("Re_b = ", ReBulk)
        
    return (xPos,waterHeightOverX, bedHeightOverX, froudeOverX, shieldsOverX, ReFricOverX)


def plotResult(result):
    fig, axes = plt.subplots(nrows=3, ncols=2,sharex=True)
    
    labels = ["$h_f$", "$h_b$", "$Fr$", "$Sh$", "$Re_\\tau$"]
    for i, ax in enumerate(axes.flatten()):
        if i >= len(labels):
            break
        ax.plot(result[0], result[i+1],'-')
        ax.set_ylabel(labels[i])
        ax.set_xlabel("$x$")
        
    fig.tight_layout()
    plt.show()

In [ ]:
with open(baseFolder+'/info.txt') as f:
    lines = f.readlines()
    evalFrequency = int(lines[0])
    gravAcceleration = float(lines[1])
    dynViscosity =  float(lines[2])
    densityRatio = float(lines[3])
    avgDiameter = float(lines[4])
    xSize = int(lines[5])
    ySize = int(lines[6])
    zSize = int(lines[7])
    numParticles = int(lines[8])

print("Eval infos:")
print(evalFrequency, gravAcceleration, dynViscosity, densityRatio, avgDiameter,numParticles)
maxTimeStep = extractMaxAvailableTimeStep()

averageVelocity = 0.02
fluidDensity = 1
kinematicViscosity = dynViscosity / fluidDensity
#timeStepSpacing = 50*evalFrequency
timeStepSpacing = 10*evalFrequency
evalTimeSteps = np.arange(0,maxTimeStep,timeStepSpacing)

lengthRef = avgDiameter
timeRef = avgDiameter / averageVelocity

print("Will evaluate", len(evalTimeSteps), "slices, from 0 to", maxTimeStep, "with spacing", timeStepSpacing)

In [ ]:
hbOverTime = np.zeros((len(evalTimeSteps), xSize))
hfOverTime = np.zeros((len(evalTimeSteps), xSize))
for i,t in enumerate(evalTimeSteps):
    result = evalTimeStep(t)
    
    # uncomment if plot per time step required
    #plotResult(result)
    
    hfAvg = np.average(result[1])
    hbAvg = np.average(result[2])
    FrAvg = np.average(result[3])
    ShAvg = np.average(result[4])
    print(f"{i}, t = {t}: hf = {hfAvg:.2f}, hb = {hbAvg:.2f}, Fr = {FrAvg:.3f}, Sh = {ShAvg:.3e}")
    hfOverTime[i] = result[1]
    hbOverTime[i] = result[2]
    

In [ ]:
# plot 2D x-t-Data evaluation in lattice units

xPlotLimits = np.array([0,xSize]) / lengthRef
tPlotLimits = np.array([min(evalTimeSteps), max(evalTimeSteps)]) / timeRef

fluidSurfaceOverTime = hbOverTime+hfOverTime
plt.figure()
plt.imshow(hbOverTime, interpolation='none', origin='lower',
           aspect='auto',extent=(*xPlotLimits,*tPlotLimits))
#plt.imshow(fluidSurfaceOverTime[:,:], interpolation='none', origin='lower',aspect='auto')
plt.colorbar()
plt.xlabel(r"$x / D$")
plt.ylabel(r"$t / t_{ref}$")
plt.show()

In [ ]:
# values taken from job output file
dx = 0.00025

# E1
dt = 1.38442e-05

# E4
#dt = 1.08125e-5

In [ ]:
# plot 2D x-t-Data evaluation in SI units

xPlotLimits = np.array([0,xSize]) * dx
tPlotLimits = np.array([min(evalTimeSteps), max(evalTimeSteps)]) * dt
averageHb = np.average(hbOverTime) * dx

#fluidSurfaceOverTime = (hbOverTime+hfOverTime) * 0.00025
plt.figure()
plt.imshow(hbOverTime[:] * dx - averageHb, interpolation='none', origin='lower',
           aspect='auto', extent=(*xPlotLimits,*tPlotLimits), vmin=-5e-3, vmax=5e-3)
#plt.imshow(fluidSurfaceOverTime[:,:], interpolation='none', origin='lower',aspect='auto')
plt.colorbar()
plt.xlabel(r"$x$ / m")
plt.ylabel(r"$t$ / s")
plt.show()

print(tPlotLimits)

In [ ]:
# plot bed height evaluation in SI units with
    # time restricted to 30 - 75 s
    # x-domain restricted to 0 - 0.75 m

timestep_length = evalTimeSteps[1] - evalTimeSteps[0]

t_min = 30
timestep_index_min = int((t_min / dt) // timestep_length)

t_max = 75
timestep_index_max = int((t_max / dt) // timestep_length)

x_min = 0
x_index_min = int(x_min // dx)

x_max = 0.75
x_index_max = int(x_max // dx)

xPlotLimits = np.array([x_min,x_max])
tPlotLimits = np.array([t_min, t_max])
averageHb = np.average(hbOverTime[timestep_index_min:timestep_index_max,x_index_min:x_index_max]) * dx

plt.figure()
plt.imshow(hbOverTime[timestep_index_min:timestep_index_max,x_index_min:x_index_max] * dx - averageHb, interpolation='none', origin='lower',
           aspect='auto', extent=(*xPlotLimits,*tPlotLimits), vmin=-5e-3, vmax=5e-3)

cb = plt.colorbar(label=r"$h$ / m")
match_colorbar(cb)
#cb.ax.tick_params(labelsize=8) 
plt.xlabel(r"$x$ / m")
plt.ylabel(r"$t$ / s")

import tikzplotlib
tikzplotlib.clean_figure()
tikzplotlib.save(
    '/home/rzlin/ca36xymo/tikz/bed-elevation-simulation.tex',
    axis_height = '\\figureheight',
    axis_width = '\\figurewidth'
    )

plt.show()

In [ ]:
print(np.average(hfOverTime[timestep_index_min:timestep_index_max,x_index_min:x_index_max]))

In [ ]:
import scipy.fft

# plot PSD with
    # time restricted to 30 - 75 s
    # x-domain restricted to 0 - 0.75 m

timestep_length = evalTimeSteps[1] - evalTimeSteps[0]

t_min = 30
timestep_index_min = int((t_min / dt) // timestep_length)

t_max = 75
timestep_index_max = int((t_max / dt) // timestep_length)

x_min = 0
x_index_min = int(x_min // dx)

x_max = 0.75
x_index_max = int(x_max // dx)

averageHb = np.average(hbOverTime[timestep_index_min:timestep_index_max,x_index_min:x_index_max]) * dx

hbOverTimeShort = hbOverTime[timestep_index_min:timestep_index_max,x_index_min:x_index_max] * dx - averageHb

# power spectral density (PSD) on period-wavelength (PW) plane
psd_pw = scipy.fft.fft2(hbOverTimeShort)
psd_pw = np.abs(psd_pw)**2 / (hbOverTimeShort.shape[0]*hbOverTimeShort.shape[1])

ft_period = np.abs(1 / scipy.fft.fftfreq(np.transpose(hbOverTimeShort).shape[-1],timestep_length)[1:] * dt)
ft_wavelength = np.abs(1 / scipy.fft.fftfreq(hbOverTimeShort.shape[-1],1)[1:] * dx)

plt.figure()
cs = plt.contour(ft_period, ft_wavelength, np.transpose(np.abs(psd_pw[1:,1:])), levels=[0.01], colors='tab:orange',linewidths=6)
plt.xlabel(r"$T$ / s")
plt.ylabel(r"$\lambda$ / m")
ax = plt.gca()
ax.set_xlim([5, 70])
ax.set_ylim([0.05, 0.2])
#cb = plt.colorbar(label=r"PSD / m$^2$")


## uncomment the following things to only store the png (no axes etc.)
## => tikz image must be created manually later
# plt.subplots_adjust(bottom = 0)
# plt.subplots_adjust(top = 1)
# plt.subplots_adjust(right = 1)
# plt.subplots_adjust(left = 0)
# plt.axis('off')
# plt.savefig('/home/rzlin/ca36xymo/tikz/spectral-density-simulation.png',bbox_inches='tight',transparent=True, pad_inches=0)

plt.show()

In [ ]:
psd_pw_new = np.transpose(np.abs(psd_pw[1:,1:]))

# compute celerity (for each point in wavelength-period PSD-array)
# => identify all possible y-axis values for new celerity-wavelength PSD-array
c = []
x = 0
for wavelength in ft_wavelength:
    y = 0
    for period in ft_period:
        if psd_pw_new[x,y] > 0.005:
            c.append(wavelength / period)
        y += 1
    x += 1
        
c = list(set(c))    # remove duplicate entries from list
c.sort()            # sort in ascending order
# list 'c' contains all possible celerities in ascending order without duplicates
# => this will be the x-axis of the celerity-wavelength PSD-array 

# create celerity-wavelength PSD-array 
psd_cw = np.zeros((ft_wavelength.shape[0], len(c)))

# fill this array with PSD at correct position
for i_wavelength in range(0, ft_wavelength.shape[0]):
    for i_period in range(0, ft_period.shape[0]):
        if psd_pw_new[i_wavelength,i_period] > 0.005:
            celerity = ft_wavelength[i_wavelength] / ft_period[i_period] # compute celerity
            i_celerity = c.index(celerity) # map celerity to it's index on the x-axis of the new array
            psd_cw[i_wavelength,i_celerity] = abs(psd_pw_new[i_wavelength,i_period]) # store PSD at correct position in new array

plt.figure()
plt.contourf(ft_wavelength, np.asarray(c), np.abs(np.transpose(psd_cw)),colors='tab:orange')
#plt.colorbar()
plt.xlabel(r"$\lambda$ / m")
plt.ylabel(r"$c$ / m/s")
ax = plt.gca()
ax.set_xlim([0.05, 0.2])
ax.set_ylim([0, 0.015])

# # uncomment the following things to only store the png (no axes etc.)
# # => tikz image must be created manually later
plt.subplots_adjust(bottom = 0)
plt.subplots_adjust(top = 1)
plt.subplots_adjust(right = 1)
plt.subplots_adjust(left = 0)
plt.axis('off')
plt.savefig('/home/rzlin/ca36xymo/tikz/celerity-simulation-e1.png',bbox_inches='tight',transparent=True, pad_inches=0)

plt.show()

In [ ]:
# FFT along x per time frame -> find wavelength

import scipy.fft

N = hbOverTime.shape[-1] # = number of samples, here xSize
colors = sns.color_palette("crest",hbOverTime.shape[0] )
plt.figure()

for i,t in enumerate(evalTimeSteps):

    yf = scipy.fft.rfft(hbOverTime[i])
    sampleSpacing = 1 # = dx
    
    # discard first element (is 0 frequency -> offset of sinus)
    xf = scipy.fft.rfftfreq(N,sampleSpacing)[1:]
    yfMod = (2.0 / N * np.abs(yf))[1:]
    plt.plot(1/xf, yfMod, color=colors[i])

    print(t, "Dominant wavelength / D = ", (1/xf[np.argmax(yfMod)])/avgDiameter )

plt.xlabel("wave length (LU)")
plt.ylabel("amplitude (LU)")
plt.show()

In [ ]:
# FFT along y per dx -> find period length (same as above with transposed array)

import scipy.fft

averageHb = np.average(hbOverTime) * dx
# use only data from some later time step to avoid including the influence of the initial condition
hbOverTimeShort = hbOverTime[:250,:] * dx - averageHb

hbOverTimeTransposed = np.transpose(hbOverTimeShort)

N = hbOverTimeTransposed.shape[-1] # = number of samples, here xSize
colors = sns.color_palette("crest", hbOverTimeTransposed.shape[0] )
plt.figure()

for i,t in enumerate(np.arange(0,hbOverTime.shape[-1])):

    yf = scipy.fft.rfft(hbOverTimeTransposed[i])
    sampleSpacing = evalTimeSteps[1] - evalTimeSteps[0]
    
    # discard first element (is 0 frequency -> offset of sinus)
    xf = scipy.fft.rfftfreq(N,sampleSpacing)[1:]
    yfMod = (2.0 / N * np.abs(yf))[1:]
    plt.plot(1/xf, yfMod, color=colors[i])

    print(t, "Dominant period length / s = ", (1/xf[np.argmax(yfMod)]) * dt )

plt.xlabel("Period length T / s")
plt.ylabel("amplitude (LU)")
plt.show()

In [ ]:
# # transport rate

# def readValueOverTimeFromFile(fileName, columnIndex):
#     return np.transpose(np.loadtxt(fileName, usecols=(0,columnIndex)))

# sedimentTransportRateData = readValueOverTimeFromFile(baseFolder+'/bedload.txt',1)


# transportRateRef = avgDiameter * averageVelocity # = l_ref^2 / t_ref,  m^2 / s

# plt.figure()
# plt.plot(sedimentTransportRateData[0] / timeRef, sedimentTransportRateData[1] / transportRateRef, '-',label="simulation")

# plt.axhline(2.0e-5 / (2.9e-3 * 0.37), linestyle='--', color='k', label="E1 (experiment)")
# plt.axhline(6.1e-5 / (2.9e-3 * 0.46), linestyle='-.', color='k', label="E4 (experiment)")
# plt.xlabel(r"$t / t_{ref}$")
# plt.ylabel(r"$q_s / q_{ref}$")
# plt.grid()
# plt.legend()
# plt.show()

# # import tikzplotlib
# # tikzplotlib.clean_figure()
# # tikzplotlib.save(
# #     'bedload-transport-rate.tex',
# #     axis_height = '\\figureheight',
# #     axis_width = '\\figurewidth'
# #     )

# # in Pascal Paper:
# # q_s,out approx q_s,in (Fig. 5)
# # -> take values from Tab. 1 as reference
# # E1: 2.0e-5 , E4: 6.1e-5 m^2/s
# # normalize by averageDiameter ( = 2.9mm) * averageVelocity (U)
# # -> E1: q_normalized = 0.0186, E4: q_normalized = 0.0457

In [ ]:
# transport rate in SI units

def readValueOverTimeFromFile(fileName, columnIndex):
    return np.transpose(np.loadtxt(fileName, usecols=(0,columnIndex)))

sedimentTransportRateData = readValueOverTimeFromFile(baseFolder+'/bedload.txt',1)

timestep_length = sedimentTransportRateData[0,1] - sedimentTransportRateData[0,0]

t_min = 30
timestep_index_min = int((t_min / dt) // timestep_length)

t_max = 75
timestep_index_max = int((t_max / dt) // timestep_length)

plt.figure()

## E1
#filepath = "/simdata/on74yces/experiment-bedload-rate/qs_E1.csv"
#exp_mean = np.array([[t_min, t_min + 10, t_max], [2.0e-5, 2.0e-5, 2.0e-5]])

# E4
filepath = "/simdata/on74yces/experiment-bedload-rate/qs_E4.csv"
exp_mean = np.array([[t_min, t_min + 10, t_max], [6.1e-5, 6.1e-5, 6.1e-5]])

data = np.transpose(np.loadtxt(filepath, delimiter=';',skiprows=1))
plt.plot(data[0,int(t_min//0.2):int(t_max//0.2)], data[1,int(t_min//0.2):int(t_max//0.2)], '-',label="Experiment")
#plt.plot(exp_mean[0], exp_mean[1], linestyle='--', color='black', label="Experiment (mean)")

plt.plot(sedimentTransportRateData[0,timestep_index_min:timestep_index_max] * dt, 
         sedimentTransportRateData[1,timestep_index_min:timestep_index_max] * dx**2 / dt, '-',label="Simulation")
mean = np.mean(sedimentTransportRateData[1,timestep_index_min:timestep_index_max] * dx**2 / dt)
mean_array = np.array([[t_min, t_min + 10, t_max], [mean, mean, mean]])
#plt.plot(mean_array[0], mean_array[1], linestyle='-', color='black', label="Simulation (mean)")

plt.xlabel(r"$t$ / s")
plt.ylabel(r"$q_s$ / (m$^{2}$/s)")

ax = plt.gca()
ax.set_xlim([t_min, t_max])

plt.grid()
#plt.legend()
plt.show()

print(np.mean(sedimentTransportRateData[1,timestep_index_min:timestep_index_max] * dx**2 / dt))

# For tikzplotlib export:
# - remove legend
# - remove mean plots
import tikzplotlib
tikzplotlib.clean_figure()
tikzplotlib.save(
    '/home/rzlin/ca36xymo/tikz/bedload-rate.tex',
    axis_height = '\\figureheight',
    axis_width = '\\figurewidth'
    )

In [ ]:
# compute median particle diameter

import numpy as  np

filepath = "/simdata/on74yces/2395774/spheres_out.dat"

particles = np.loadtxt(filepath, delimiter=' ', skiprows=1)

# 50th percentile diameter
diameterMedian = np.median(particles[:,4]*2)
print("50th percentile, i.e., median particle diameter: ", diameterMedian, "m")
print("50th percentile, i.e., median particle diameter: ", diameterMedian / dx, "cells")

# 16th percentile diameter
diameter16th = np.percentile(particles[:,4]*2, 16)
print("16th percentile particle diameter: ", diameter16th, "m")
print("16th percentile particle diameter: ", diameter16th / dx, "cells")

# 84th percentile diameter
diameter84th = np.percentile(particles[:,4]*2, 84)
print("84th percentile particle diameter: ", diameter84th, "m")
print("84th percentile particle diameter: ", diameter84th / dx, "cells")

In [ ]:
# plot data from experiment of Pascal et al.
import math

filepath = "/simdata/on74yces/experiment-bed-elevation/bed_surface_in_time_E4.csv"
data = np.loadtxt(filepath, delimiter=';')

# crop data to relevant time steps
data = data[1200:1450,:]

# remove inclination from data
sine3degree = math.sin(3.0*math.pi/180)
for x in range(0, data.shape[1]):
    data[:,x] = data[:,x] - abs(x-1280)*0.75/data.shape[1]*sine3degree

xPlotLimitsN = np.array([0, data.shape[1]]) * 0.75/data.shape[1]
tPlotLimitsN = np.array([1200, 1200+data.shape[0]])
averageHbN = np.average(data)

plt.figure()
plt.imshow(data - averageHbN, interpolation='none', origin='lower',
           aspect='auto', extent=(*xPlotLimitsN,*tPlotLimitsN), vmin=-5e-3, vmax=5e-3)
plt.colorbar()
plt.xlabel(r"$x$ / m")
plt.ylabel(r"$t$ / s")
plt.show()

print(data.shape)

In [ ]:
import math

# plot bed height evaluation of experiment
    # time restricted to 1200 - 1245 s
    # x-domain restricted to 0 - 0.75 m
    
filepath = "/simdata/on74yces/experiment-bed-elevation/bed_surface_in_time_E1.csv"
hbOverTimeExp = np.loadtxt(filepath, delimiter=';')

dxExp = 0.75 / 1280

# remove inclination from data
sine3degree = math.sin(3*math.pi/180)
for x in range(0, hbOverTimeExp.shape[1]):
    hbOverTimeExp[:,x] = hbOverTimeExp[:,x] - abs(x-1280)*0.75/hbOverTimeExp.shape[1]*sine3degree

t_min = 1200
timestep_index_min = t_min

t_max = 1245
timestep_index_max = t_max

x_min = 0
x_index_min = int(x_min // dxExp)

x_max = 0.75
x_index_max = int(x_max // dxExp)

xPlotLimits = np.array([x_min,x_max])
tPlotLimits = np.array([t_min, t_max])
averageHbExp = np.average(hbOverTimeExp)

plt.figure()
plt.imshow(hbOverTimeExp[timestep_index_min:timestep_index_max,x_index_min:x_index_max] - averageHbExp, 
           interpolation='none', origin='lower', aspect='auto', extent=(*xPlotLimits,*tPlotLimits), 
           vmin=-5e-3, vmax=5e-3)

cb = plt.colorbar(label=r"$h$ / m")
#match_colorbar(cb)
#cb.ax.tick_params(labelsize=8) 
plt.xlabel(r"$x$ / m")
plt.ylabel(r"$t$ / s")

import tikzplotlib
tikzplotlib.clean_figure()
tikzplotlib.save(
    '/home/rzlin/ca36xymo/tikz/bed-elevation-experiment.tex',
    axis_height = '\\figureheight',
    axis_width = '\\figurewidth'
    )

plt.show()

In [ ]:
import scipy.fft

# plot spectral density of bed height in experiment

filepath = "/simdata/on74yces/experiment-bed-elevation/bed_surface_in_time_E1.csv"
hbOverTimeExp = np.loadtxt(filepath, delimiter=';')

dxExp = 0.75 / 1280

# remove inclination from data
sine3degree = math.sin(2.9*math.pi/180)
for x in range(0, hbOverTimeExp.shape[1]):
    hbOverTimeExp[:,x] = hbOverTimeExp[:,x] - abs(x-1280)*0.75/hbOverTimeExp.shape[1]*sine3degree

averageHbExp = np.average(hbOverTimeExp)

# power spectral density (PSD) on period-wavelength (PW) plane
psd_pw_exp = scipy.fft.fft2(hbOverTimeExp - averageHbExp)
psd_pw_exp = np.abs(psd_pw_exp)**2 / (hbOverTimeExp.shape[0] * hbOverTimeExp.shape[1])

ft_period_exp = np.abs(1 / scipy.fft.fftfreq(np.transpose(hbOverTimeExp).shape[-1],1)[1:])
ft_wavelength_exp = np.abs(1 / scipy.fft.fftfreq(hbOverTimeExp.shape[-1],0.75/1280)[1:])

plt.figure()

plt.contourf(ft_period_exp, ft_wavelength_exp, np.transpose(np.abs(psd_pw_exp[1:,1:])))
#cb = plt.colorbar(label=r"PSD / m$^2$")
#match_colorbar(cb)
plt.xlabel(r"$T$ / s")
plt.ylabel(r"$\lambda$ / m")
ax = plt.gca()
ax.set_xlim([5, 70])
ax.set_ylim([0.05, 0.2])

# uncomment the following things to only store the png (no axes etc.)
# => tikz image must be created manually later
plt.subplots_adjust(bottom = 0)
plt.subplots_adjust(top = 1)
plt.subplots_adjust(right = 1)
plt.subplots_adjust(left = 0)
plt.axis('off')
plt.savefig('/home/rzlin/ca36xymo/tikz/spectral-density-experiment.png',bbox_inches='tight',transparent=True, pad_inches=0)


plt.show()

In [ ]:
psd_pw_exp_new = np.transpose(np.abs(psd_pw_exp[1:,1:]))

# compute celerity (for each point in wavelength-period PSD-array)
# => identify all possible y-axis values for new celerity-wavelength PSD-array
c_exp = []
x = 0
for wavelength in ft_wavelength_exp:
    y = 0
    for period in ft_period_exp:
        if psd_pw_exp_new[x,y] > 0.0001:
            c_exp.append(wavelength / period)
        y += 1
    x += 1

c_exp = list(set(c_exp))    # remove duplicate entries from list
c_exp.sort()                # sort in ascending order
# list 'c' contains all possible celerities in ascending order without duplicates
# => this will be the x-axis of the celerity-wavelength PSD-array

# create celerity-wavelength PSD-array 
psd_cw = np.zeros((ft_wavelength_exp.shape[0], len(c_exp)))

# fill this array with PSD at correct position
for i_wavelength in range(0, ft_wavelength_exp.shape[0]):
    for i_period in range(0, ft_period_exp.shape[0]):
        if psd_pw_exp_new[i_wavelength,i_period] > 0.0001:
            celerity = ft_wavelength_exp[i_wavelength] / ft_period_exp[i_period] # compute celerity
            i_celerity = c_exp.index(celerity) # map celerity to it's index on the x-axis of the new array
            psd_cw[i_wavelength,i_celerity] = abs(psd_pw_exp_new[i_wavelength,i_period]) # store PSD at correct position in new array

plt.figure()
plt.contourf(ft_wavelength_exp, np.asarray(c_exp), np.abs(np.transpose(psd_cw)),colors='tab:blue')
#plt.colorbar()
plt.xlabel(r"$\lambda$ / m")
plt.ylabel(r"$c$ / m/s")
ax = plt.gca()
ax.set_xlim([0.05, 0.2])
ax.set_ylim([0, 0.015])

# uncomment the following things to only store the png (no axes etc.)
# => tikz image must be created manually later
plt.subplots_adjust(bottom = 0)
plt.subplots_adjust(top = 1)
plt.subplots_adjust(right = 1)
plt.subplots_adjust(left = 0)
plt.axis('off')
plt.savefig('/home/rzlin/ca36xymo/tikz/celerity-experiment-e1.png',bbox_inches='tight',transparent=True, pad_inches=0)


plt.show()

In [ ]:
# grid refinement study: transport rate in SI units

def readValueOverTimeFromFile(fileName, columnIndex):
    return np.transpose(np.loadtxt(fileName, usecols=(0,columnIndex)))

t_min = 0
t_max = 50
t_min_for_mean = 10

dx_coarse = 0.0005
dt_coarse = 2.16294e-05
sedimentTransportRateData_coarse = readValueOverTimeFromFile("/simdata/on74yces/resolution5"+'/bedload.txt',1)
timestep_length_coarse = sedimentTransportRateData_coarse[0,1] - sedimentTransportRateData_coarse[0,0]
timestep_index_min_coarse = int((t_min / dt_coarse) // timestep_length_coarse)
timestep_index_max_coarse = int((t_max / dt_coarse) // timestep_length_coarse)
timestep_index_min_mean_coarse = int((35 / dt_coarse) // timestep_length_coarse)
mean_coarse = np.mean(sedimentTransportRateData_coarse[1,timestep_index_min_mean_coarse:timestep_index_max_coarse] * dx_coarse**2 / dt_coarse)
mean_array_coarse = np.array([[35, 35+1, t_max], [mean_coarse, mean_coarse, mean_coarse]])

print(sedimentTransportRateData_coarse[0,-1])
print(sedimentTransportRateData_coarse[0,-1] * dt_coarse)
print(mean_coarse)

dx_medium = 0.00025
dt_medium = 1.08147e-05
sedimentTransportRateData_medium = readValueOverTimeFromFile("/simdata/on74yces/resolution10"+'/bedload.txt',1)
timestep_length_medium = sedimentTransportRateData_medium[0,1] - sedimentTransportRateData_medium[0,0]
timestep_index_min_medium = int((t_min / dt_medium) // timestep_length_medium)
timestep_index_max_medium = int((t_max / dt_medium) // timestep_length_medium)
timestep_index_min_mean_medium = int((t_min_for_mean / dt_medium) // timestep_length_medium)
mean_medium = np.mean(sedimentTransportRateData_medium[1,timestep_index_min_mean_medium:timestep_index_max_medium] * dx_medium**2 / dt_medium)
mean_array_medium = np.array([[t_min_for_mean, t_min_for_mean+1, t_max], [mean_medium, mean_medium, mean_medium]])

print(sedimentTransportRateData_medium[0,-1])
print(sedimentTransportRateData_medium[0,-1] * dt_medium)
print(mean_medium)

dx_fine = 0.000125
dt_fine = 5.40734e-06
sedimentTransportRateData_fine = readValueOverTimeFromFile("/simdata/on74yces/resolution20"+'/bedload.txt',1)
timestep_length_fine = sedimentTransportRateData_fine[0,1] - sedimentTransportRateData_fine[0,0]
timestep_index_min_fine = int((t_min / dt_fine) // timestep_length_fine)
timestep_index_max_fine = int((t_max / dt_fine) // timestep_length_fine)
timestep_index_min_mean_fine = int((t_min_for_mean / dt_fine) // timestep_length_fine)
mean_fine = np.mean(sedimentTransportRateData_fine[1,timestep_index_min_mean_fine:timestep_index_max_fine] * dx_fine**2 / dt_fine)
mean_array_fine = np.array([[t_min_for_mean, t_min_for_mean+1, t_max], [mean_fine, mean_fine, mean_fine]])

print(sedimentTransportRateData_fine[0,-1])
print(sedimentTransportRateData_fine[0,-1] * dt_fine)
print(mean_fine)

plt.figure()
plt.plot(sedimentTransportRateData_coarse[0,timestep_index_min_coarse:timestep_index_max_coarse:4] * dt_coarse,
         sedimentTransportRateData_coarse[1,timestep_index_min_coarse:timestep_index_max_coarse:4] * dx_coarse**2 / dt_coarse,
         '-',label="Coarse",color="tab:purple")

plt.plot(sedimentTransportRateData_medium[0,timestep_index_min_medium:timestep_index_max_medium:8] * dt_medium,
         sedimentTransportRateData_medium[1,timestep_index_min_medium:timestep_index_max_medium:8] * dx_medium**2 / dt_medium,
         '-',label="Medium")

plt.plot(sedimentTransportRateData_fine[0,timestep_index_min_fine:timestep_index_max_fine:16] * dt_fine,
         sedimentTransportRateData_fine[1,timestep_index_min_fine:timestep_index_max_fine:16] * dx_fine**2 / dt_fine,
         '-',label="Fine")

plt.plot(mean_array_coarse[0], mean_array_coarse[1], linestyle=':', color='black', label="Coarse (mean)")
plt.plot(mean_array_medium[0], mean_array_medium[1], linestyle='-', color='black', label="Medium (mean)")
plt.plot(mean_array_fine[0], mean_array_fine[1], linestyle='--', color='black', label="Fine (mean)")

plt.xlabel(r"$t$ / s")
plt.ylabel(r"$q_s$ / (m$^{2}$/s)")

ax = plt.gca()
ax.set_xlim([t_min, t_max])

plt.grid()
plt.legend()
plt.show()

# # For tikzplotlib export:
# # - remove legend
# # - remove mean plots
# import tikzplotlib
# tikzplotlib.clean_figure()
# tikzplotlib.save(
#     '/home/rzlin/ca36xymo/tikz/resolution-study.tex',
#     axis_height = '\\figureheight',
#     axis_width = '\\figurewidth'
#     )

In [ ]:
def readValueOverTimeFromFile(fileName, columnIndex):
    return np.loadtxt(fileName, usecols=(0,columnIndex))

dx = 0.00025
dt = 1.08147e-05
velocity = readValueOverTimeFromFile("/simdata/on74yces/merge_E4_flat_new"+'/fluidInfo.txt',2)
t = velocity[:,0] * dt
velocity = velocity[:,1] * dx/dt
plt.plot(t, velocity)

plt.ylabel(r"$U_{x,\text{l}}$ / (m/s)")
plt.xlabel(r"$t$ / s")

import tikzplotlib
tikzplotlib.clean_figure()
tikzplotlib.save(
    '/home/rzlin/ca36xymo/tikz/bed-elevation-simulation.tex',
    axis_height = '\\figureheight',
    axis_width = '\\figurewidth'
    )

plt.show()

In [ ]:
# old stuff below

In [ ]:
(3*0.37 / 0.0083 * 1e-6)/((2.55-1)*9.81*2.9e-3)

In [ ]:
print(np.max(hbOverTime[0,:]),np.min(hbOverTime[0,:]))

In [ ]:
import seaborn as sns
plt.figure()
sns.distplot(x=fluidSurfaceOverTime[0,:])
plt.show()